In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qtpyt.base.greenfunction import GreenFunction
from qtpyt.base.leads import LeadSelfEnergy
from qtpyt.parallel import comm
from qtpyt.parallel.egrid import GridDesc
from qtpyt.surface.tools import prepare_leads_matrices
from qtpyt.tools import expand_coupling

In [2]:
H_device = np.load("hamiltonian/H_R_device.npy")
H_device = H_device.astype(np.complex128)
S_device = np.eye(H_device.shape[0])
H_k_leads = np.load("hamiltonian/H_k_leads.npy")
nkpts = H_k_leads.shape[0]
dim = H_k_leads.shape[1]
S_k_leads = np.zeros((nkpts, dim, dim), dtype=np.complex128)
for i in range(nkpts):
    S_k_leads[i] = np.eye(dim, dtype=np.complex128)

In [3]:
H_k_leads.shape
# (nkpts, dim, dim) where nkpts is the number of k-points in the irreducible Brillouin zone, dim is the number of orbitals * number of atoms in the lead. Here we have 72 atoms in the lead and 1 orbital per atom (2pz), so dim = 72.

S_k_leads.shape
# We assume overlap matrix is identity for the leads, so S_k_leads is just an identity matrix of the same shape as H_k_leads.

(6, 72, 72)

In [4]:
nkpts_leads = (6, 1, 1)

# Prepare the k-points and matrices for the leads (Hamiltonian and overlap matrices)
kpts_t, h_leads_kii, s_leads_kii, h_leads_kij, s_leads_kij = map(lambda m: m[0], prepare_leads_matrices(
    H_k_leads,
    S_k_leads,
    nkpts_leads,
    align=(0, H_device[0, 0]),
))


In [5]:
print(kpts_t) # kpts transverse to the transport direction (transport direction is along the x-axis here)
print(h_leads_kii.shape) # Hamiltonian matrix for the leads in the k-space
print(h_leads_kij.shape)

[0. 0. 0.]
(72, 72)
(72, 72)


In [ ]:
# Initialize self-energy list for left and right leads
self_energy = [None, None]

# Create LeadSelfEnergy objects for left and right leads
# Uses Sancho Rubio method to compute the surface Green's function
self_energy[0] = LeadSelfEnergy((h_leads_kii, s_leads_kii), (h_leads_kij, s_leads_kij))
self_energy[1] = LeadSelfEnergy(
    (h_leads_kii, s_leads_kii), (h_leads_kij, s_leads_kij), id="right"
)

print(self_energy[0].shape)

(72, 72)


In [ ]:
# expand dimension of lead self energy to dimension of scattering region
expand_coupling(self_energy[0], len(H_device[0]))
expand_coupling(self_energy[1], len(H_device[0]), id='right')
print(H_device.shape)
print(self_energy[0].shape)


(4800, 4800)
(4800, 4800)


In [8]:
de = 0.1
energies = np.arange(-1.,1.+de/2.,de).round(7)
eta = 1e-3

# slice(None) means that we've already expanded the leads to the scattering region
gf = GreenFunction(H_device, S_device, selfenergies=[(slice(None),self_energy[0]),(slice(None),self_energy[1])], eta=eta)


In [9]:
gf.H.shape

(4800, 4800)

In [ ]:
gd = GridDesc(energies, 1)
T = np.empty(gd.energies.size)

for e, energy in enumerate(gd.energies):
    T[e] = gf.get_transmission(energy)

T = gd.gather_energies(T)
if comm.rank == 0:
    np.save(f'ET', (energies, T))

gf.Ginv [[-1. +0.001j  2.7+0.j    -0. +0.j    ... -0. +0.j    -0. +0.j
  -0. +0.j   ]
 [ 2.7+0.j    -1. +0.001j  2.7+0.j    ... -0. +0.j    -0. +0.j
  -0. +0.j   ]
 [-0. +0.j     2.7+0.j    -1. +0.001j ... -0. +0.j    -0. +0.j
  -0. +0.j   ]
 ...
 [-0. +0.j    -0. +0.j    -0. +0.j    ... -1. +0.001j  2.7+0.j
  -0. +0.j   ]
 [-0. +0.j    -0. +0.j    -0. +0.j    ...  2.7+0.j    -1. +0.001j
   2.7+0.j   ]
 [-0. +0.j    -0. +0.j    -0. +0.j    ... -0. +0.j     2.7+0.j
  -1. +0.001j]]
gf.Ginv [[-0.9+0.001j  2.7+0.j    -0. +0.j    ... -0. +0.j    -0. +0.j
  -0. +0.j   ]
 [ 2.7+0.j    -0.9+0.001j  2.7+0.j    ... -0. +0.j    -0. +0.j
  -0. +0.j   ]
 [-0. +0.j     2.7+0.j    -0.9+0.001j ... -0. +0.j    -0. +0.j
  -0. +0.j   ]
 ...
 [-0. +0.j    -0. +0.j    -0. +0.j    ... -0.9+0.001j  2.7+0.j
  -0. +0.j   ]
 [-0. +0.j    -0. +0.j    -0. +0.j    ...  2.7+0.j    -0.9+0.001j
   2.7+0.j   ]
 [-0. +0.j    -0. +0.j    -0. +0.j    ... -0. +0.j     2.7+0.j
  -0.9+0.001j]]


In [ ]:
plt.plot(energies, T)
plt.xlabel("Energy (eV)")
plt.ylabel("Transmission")
plt.title("Transmission vs Energy")
plt.grid()
plt.show()
